In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv('../input/samsung-electronics-stock-historical-price/005930.KS.csv')
df

In [ ]:
fig = make_subplots(specs=[[{'secondary_y':True}]])
fig.add_trace(
    go.Scatter(x=df.Date,y=df.Close,name='Close')
)
fig.add_trace(
    go.Scatter(x=df.Date,y=df['Adj Close'],name='Adj Close')
)
fig.update_layout(xaxis_title='Date',
                  yaxis_title='Price'
                 )
fig.show()

In [ ]:
def bolinger(data):
    data['MA20'] = data.Close.rolling(window=20).mean()
    data['std'] = data.Close.rolling(window=20).std()
    data['upper_20'] = data.MA20 + 2 * data['std']
    data['lower_20'] = data.MA20 - 2 * data['std']
    data.drop('std',axis=1,inplace=True)
    data.dropna(inplace=True)
    return data

In [ ]:
665 % (len(df)-1)

In [ ]:
df = bolinger(df)
df.ffill(inplace=True)
df.reset_index(drop=True,inplace=True)
df

In [ ]:
fig = make_subplots(specs=[[{'secondary_y':True}]])
fig.add_trace(
    go.Scatter(x=df.Date,y=df.Close,name='Close')
)
fig.add_trace(
    go.Scatter(x=df.Date,y=df.upper_20,name='Upper 20')
)
fig.add_trace(
    go.Scatter(x=df.Date,y=df.lower_20,name='Lower 20')
)
fig.update_layout(title='ploting bolinger band',
                  xaxis_title='Date',
                  yaxis_title='Price'
                 )
fig.show()

# Invest Strategy
* if Close meet upper band or similar my backtesting function ---> sell stock
* if Close meet lower band or similar my backtesting function ---> buy stock

In [ ]:
import math

In [ ]:
total_balance = 1000000
def backtesting(df,total_balance):
    buy_price = []
    buy_date = []
    sell_price = []
    sell_date = []

    long_earn = []

    num_bought = 0
    df_balance = []
    df_date = []
    balance_date = []
    trade_num = 0
    buy_num = -1
    sell_num = -1
    trade_num = -1
    position = 'none'
    for i in range(len(df.Close)):
        if total_balance <= 0:
            print('you lost all money...')
            break
        # 포지션이 없을때 진입 타이밍(볼린저 밴드)
        elif position == 'none':
            if df.Close[i] <= df['lower_20'][i] and abs(df.Close[i] - df['lower_20'][i]) <= 1000:
                position = 'long'
                buy_price.append(df.Close[i])
                buy_date.append(df.Date[i])
                if trade_num >= 0:
                    total_balance = df_balance[trade_num]
                num_bought = 0
                num_count = math.floor(total_balance / df.Close[i])
                num_bought += num_count
                remain = total_balance - (df.Close[i] * num_count)
                df_balance.append(total_balance)
                df_date.append(df.Date[i])
                print('----------------------------------------------------------------------------')
                print(f'Meet Lower Bolinger Band So Enter the position:{position}')
                print(f'{df.Date[i]}\tBuy price:{df.Close[i]}\tNum bought:{num_bought}')
                print(f'Reamin Balance:{remain}')
                print('----------------------------------------------------------------------------')
#                 temp_price = df.vwap[i]
#                 temp_time = df.time[i]
                trade_num += 1
                buy_num += 1
#             else:
#                 temp_price = df.vwap[i]
#                 temp_symbol = df.symbol[i]
#                 temp_time = df.time[i]
        elif position == 'long':
            if df.Close[i] >= df['upper_20'][i] and abs(df.Close[i] - df['upper_20'][i]) <= 1000:
                position = 'none'
                sell_price.append(df.Close[i])
                sell_date.append(df.Date[i])
                total_balance = remain
                earn = df.Close[i] - buy_price[buy_num]
                total_earn = (earn * num_bought) + (buy_price[buy_num] * num_bought)
                
                total_balance += total_earn
                df_balance.append(total_balance)
                df_date.append(df.Date[i])
                print('----------------------------------------------------------------------------')
                print(f'Meet Upper Bolinger Band So Sell the stock')
                print(f'{df.Date[i]}\tBuy price:{buy_price[buy_num]}\tSell price:{df.Close[i]}')
                print(f'Remain Balance:{total_earn}')
                print('----------------------------------------------------------------------------')
                sell_num += 1
                trade_num += 1
#                 temp_price = df.Close[i]
#                 temp_time = df.Date[i]
#             else:
        
#                 temp_price = df.Close[i]
#                 temp_time = df.Date[i]
        if i % (len(df) - 1) == 0:
            if position == 'long':
                sell_price.append(df.Close[i])
                sell_date.append(df.Date[i])
                total_balance = remain
                earn = df.Close[i] - buy_price[buy_num]
                total_earn = (earn * num_bought) + (buy_price[buy_num] * num_bought)
                total_balance += total_earn
                df_balance.append(total_balance)
                df_date.append(df.Date[i])
                print('----------------------------------------------------------------------------')
                print(f'End backtesing so sell the stock')
                print(f'{df.Date[i]}\tBuy price:{buy_price[buy_num]}\tSell price:{df.Close[i]}')
                print(f'Remain Balance:{total_earn}')
                print('----------------------------------------------------------------------------')
            elif position == 'none':
                print('End Backtesting')
    print(f'Total Trading Num:{trade_num}')
    balance = pd.DataFrame({'Date': df_date,'balance':df_balance })
    final_return_ratio = ((balance.iloc[-1][1] / 1000000) - 1) * 100
    print('Return Ratio:{:.3f}%'.format(final_return_ratio))
    print(i)
    return balance

In [ ]:
df_balance = backtesting(df,total_balance)

In [ ]:
fig = make_subplots()
fig.add_trace(
    go.Scatter(x=df_balance.Date,y=df_balance.balance,name='your Balance')
)
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='your Balance'
)
fig.show()